In [483]:
from numpy import random
from scipy import stats
import pandas as pd

- We declared the predefined variables

In [484]:
patient_per_hour = 1
S = 3
Triage_nurse_rate = 0.476190476
p_1 = 0.2
p_2 = 1 - 0.2
K = 5
Healing_in_hospital_rate = 0.213333333
healing_of_stable_rate = 0.16
random.seed(45)

- This part is the definition of the variables that we will use in the code 

In [485]:
nurses = [{'arrival':0,'departure':0,'availability':True} for i in range(0,S)]
beds = [{'departure':0,'availability':True}for i in range(0, K)]
rejected_critical_patients = 0
nurses_work_times = [0 for i in range(S)]
beds_work_times = [ 0 for i in range(K)] 
duration_homecare = 0
queue_nurse = []
time_of_simulation, event_count, total_healed, total_sick, total_in_hospital, total_homesick = 0,0,0,0,0,0
arrived_patient, last_arrived, total_triage_time, total_critical, total_time_healed = 0,0,0,0,0

- This functions executes the arrival of a patient. It appends patient to the queue first and if any bed is available, create ` Arrival Nurse ` event

In [486]:
def Execute_Arrival(id):
  queue_nurse.append({'id':id,'arrival_time':time_of_simulation,'departure_time':0})
  if check_nurse_availability():
    event_list.append({ 'id': id,'time': time_of_simulation,'type': 'AN' })


- If a patient arrives the nurse, nurse becomes unavailable and patient creates a new event ` Departure Nurse `

In [487]:
def Execute_Arrival_Nurse():
  patient = queue_nurse.pop(0)
  service_time = Retrieve_Nurse_Service_Time()
  departure_time = service_time + time_of_simulation
  available_nurse = check_nurse_availability()-1
  nurses[available_nurse]['departure'] = departure_time
  nurses[available_nurse]['availability'] = False
  nurses_work_times[available_nurse] +=service_time 
  event_list.append({'id':patient['id'],'time':departure_time, 'type':'DN'})


- While departing nurse decides the condition, 

In [488]:
def Departure_Triage(patient):
  condition = Retrieve_Condition()
  global nurses, rejected_critical_patients, total_critical
  nurses = [{'departure':nurse['departure'],'availability':time_of_simulation >= nurse['departure']} for nurse in nurses]
  if condition == 's':
    event_list.append({ 'id': patient,'time': time_of_simulation,'type': 'AH', 'condition': 's' })
  else:
    total_critical += 1
    if(check_bed_availability()):
      event_list.append({ 'id': patient,'time': time_of_simulation,'type': 'AB' })
    else:
      rejected_critical_patients += 1
      event_list.append({ 'id': patient,'time': time_of_simulation,'type': 'AH' , 'condition': 'c'})

  if (check_nurse_availability()):
    if(len(queue_nurse)):
      event_list.append({ 'id': patient,'time': time_of_simulation,'type': 'AN' })


- Patients arrive home and system creates future event of departure.

In [489]:
def Execute_Arrival_Home(patient,condition):
  global total_homesick
  global duration_homecare, total_time_healed
  total_homesick += 1
  duration_homecare +=1
  if (condition == 's'):
    home_healed_time = Retrieve_Home_Healing_Time('s')
    departure_from_heal = home_healed_time + time_of_simulation
    total_time_healed += home_healed_time
    event_list.append({'id':patient,'time':departure_from_heal, 'type':'DH'})
  else:
    departure_from_heal = Retrieve_Home_Healing_Time('c') + time_of_simulation
    total_time_healed += departure_from_heal - time_of_simulation
    event_list.append({ 'id': patient,'time': departure_from_heal,'type': 'DH' })



- Patient arrives the bed, bed is not available anymore.

In [490]:
def Execute_Arrival_Bed(patient):
  global total_time_healed
  bed_healed = Generate_Hospital_Healing_Time()
  departure_from_heal =  bed_healed + time_of_simulation
  bed_number = check_bed_availability()-1
  beds[bed_number]['departure'] = departure_from_heal
  beds[bed_number]['availability'] = False
  beds_work_times[bed_number] = bed_healed
  total_time_healed += bed_healed
  event_list.append({ 'id': patient,'time': departure_from_heal,'type': 'DB' })


- Patient deprts from the homecase, is healed now.

In [491]:
def Execute_Departure_HomeCare(patient):

  global total_healed, total_homesick, duration_homecare
  total_healed += 1
  total_homesick -= 1
  duration_homecare -= 1
  

- This functions generates the values by using random library from numpy.

In [492]:
def Generate_interarrival():
  return round(random.exponential(scale=float(60)),0) 

def Generate_Nurse_Service_Time():
  return round(random.exponential(scale=float(60/Triage_nurse_rate)),0)

def Generate_Condition():
  number = triage_result.pop(0)
  return 's' if number <= p_1  else 'c' 

def Generate_Hospital_Healing_Time():
  return round(random.exponential(scale=float(60/Healing_in_hospital_rate)),0)

def Generate_Home_Healing_Time(type):
  stable = round(random.exponential(scale=float(60/healing_of_stable_rate)),0)
  if type=='s':
    return stable
  else:
    q=random.rand()
    uniform_dist = stats.uniform.ppf(q,loc=1.75,scale=1.25)
    return round(random.exponential(scale=float(60/((1+uniform_dist) * Healing_in_hospital_rate))),0) 

We used this function to find each arrival time of patient by summing interarrivals. 

In [493]:
def Arrival():
  global last_arrived, arrived_patient
  arrived_patient+=1
  last_arrived += Retrieve_interarrival()
  return last_arrived

Retrieve the random variables from random list

In [494]:
def Retrieve_interarrival():
  return interarrival_list.pop(0)

def Retrieve_Nurse_Service_Time():
  return nurse_service_time.pop(0)

def Retrieve_Condition():
  number = triage_result.pop(0)
  return 's' if number <= p_1  else 'c' 

def Retrieve_Hospital_Healing_Time():
  return hospital_healing_time.pop(0)

def Retrieve_Home_Healing_Time(type):
  if type=='s':
    return home_healing_stable.pop(0)
  else:
    return home_healing_critical.pop(0)


Patient is healed and bed is available now.

In [495]:
def Treated_at_Hospital(patient):
  global beds
  global total_healed
  total_healed+=1
  beds = [{'departure':bed['departure'], 'availability':time_of_simulation >= bed['departure']} for bed in beds]
  

- It sets   event time to the simulation clock.

In [496]:
def Advance_Time(time):
  global time_of_simulation
  time_of_simulation = time


- We sort the events according to time and this function calls the function according the event type.

In [497]:
def Execute_Event(event):
  time = event['time']
  Advance_Time(time)
  if event['type'] == 'A':
    Execute_Arrival(event['id'])
  elif event['type'] == 'AN':
    Execute_Arrival_Nurse()
  elif event['type'] == 'AH':
    Execute_Arrival_Home(event['id'], event['condition'])
  elif event['type'] == 'AB':
    Execute_Arrival_Bed(event['id'])
  elif event['type'] == 'DN':
    Departure_Triage(event['id'])
  elif event['type'] == 'DH':
    Execute_Departure_HomeCare(event['id'])
  elif event['type'] == 'DB':
    Treated_at_Hospital(event['id'])

This two function checks the availability of beds and nurses.

In [498]:
def check_nurse_availability():
  for i in range(len(nurses)):
    if nurses[i]['availability']:
      return i+1
  return 0 

def check_bed_availability():
  for i in range(len(beds)):
    if beds[i]['availability']:
      return i+1
  return 0

- If you want to run the simulation half full you can run this code block.

In [499]:
isHalf = False
isFull = True
if isFull:
  (queue_nurse.append({'id':0,'arrival_time':time_of_simulation,'departure_time':0}) for i in range(S))
  (Execute_Arrival_Nurse() for i in range(S))
  (Execute_Arrival_Bed(1) for i in range(K))

if isHalf:
  (queue_nurse.append({'id':0,'arrival_time':time_of_simulation,'departure_time':0}) for i in range(round(S/2)))
  (Execute_Arrival_Nurse() for i in range(round(S/2)))
  (Execute_Arrival_Bed(1) for i in range(round(K/2)))


- We create random number list for each event, so that the code uses the same random variables with hand simulation,
- `while ` loop decides how many operations will be done
- Results.csv file is the table that indicates the steps taken in the simulation.
- I have printed the average values in command line.

In [500]:
interarrival_list = [ Generate_interarrival() for i in range(0,10000)] 
event_list = [ ({'id':arrived_patient, "time":Arrival(), 'type':'A'}) for i in range(0,10000) ]

nurse_service_time = [ Generate_Nurse_Service_Time() for i in range(0,10000)]
hospital_healing_time = [Generate_Hospital_Healing_Time() for i in range(0,10000)]
triage_result = [round(random.rand(),2) for i in range(0,10000)]
home_healing_stable = [Generate_Home_Healing_Time('s') for i in range(0,10000)]
home_healing_critical = [Generate_Home_Healing_Time('c') for i in range(0,10000)]
simulation_table = []
total_bed_time = 0
total_nurse_time = 0
average_patient_home = 0

while total_healed < 2000:
  duration_homecare = 0 
  earlier_date = time_of_simulation
  event_list.sort(key=lambda x: (x['time'],x['id']),reverse = True)
  event = event_list.pop()
  time_of_simulation = event['time']
  Execute_Event(event)
  total_bed = sum([ 0 if i['availability'] else 1 for i in beds])
  total_bed_time += (time_of_simulation - earlier_date)*total_bed
  total_nurse =  sum([ 0 if i['availability'] else 1 for i in nurses])
  total_nurse_time += total_nurse*(time_of_simulation - earlier_date)
  total_sick = len(queue_nurse) + total_bed + total_nurse + total_homesick 
  simulation_table.append({ 'time':time_of_simulation, 'total Sick':total_sick, 'total in hospital':total_sick - total_homesick, 'total in queue': len(queue_nurse), 'total in bed': total_bed ,'total in nurse':total_nurse, 'total healed': total_healed}| event)
  event_count += 1
  average_patient_home += duration_homecare if duration_homecare >0 else 0 * (time_of_simulation - earlier_date) 

simulation_table_df = pd.DataFrame(simulation_table)
simulation_table_df.to_csv('Results.csv')
bed_work = (float(sum(beds_work_times))/K )
nurse_work = (float(sum(nurses_work_times))/S)
kesisim = (bed_work* nurse_work)/ (time_of_simulation)**2
joint_prob_empty = (bed_work + nurse_work)/float(time_of_simulation) - kesisim
print('union of two probabilities', joint_prob_empty)
print('intersection of two probabilities',kesisim)
rejected = rejected_critical_patients/total_critical
print('rejected critical',rejected)
utilization_nurse = nurse_work/float(time_of_simulation)
print('utilization nurse',utilization_nurse)
print('average hour home',average_patient_home/time_of_simulation)
print('healed average', total_time_healed/total_healed)

union of two probabilities 0.7305824512090483
intersection of two probabilities 0.002445084548862959
rejected critical 0.16865203761755485
utilization nurse 0.7296766204616034
average hour home 0.005677101240241172
healed average 262.4165
